In [4]:
!pip install dask[dataframe]

     |████████████████████████████████| 136 kB 4.9 MB/s 


In [8]:
'''
from IPython.display import display
from pyspark.conf import SparkConf
from pyspark.sql import SparkSession
from pyspark.context import SparkContext

from IPython.display import HTML, display
import tabulate
'''

import pandas as pd
import numpy as np
import re

from dask import dataframe as dd
from google.colab import files

pd.options.display.max_colwidth = 100

# 1. Preparação dos dados

In [9]:
# Portfolio Clientes DataFrame
clientes_df = dd.read_csv("/content/drive/MyDrive/Colab Datasets/Stone Data Challenge 2022/raw/portfolio_clientes.csv")

In [26]:
# Portfolio Comunicados DataFrame
comunicados_df = dd.read_csv("/content/drive/MyDrive/Colab Datasets/Stone Data Challenge 2022/raw/portfolio_comunicados.csv")

In [11]:
# Portfolio Geral DataFrame
geral_df = dd.read_csv("/content/drive/MyDrive/Colab Datasets/Stone Data Challenge 2022/raw/portfolio_geral.csv")

In [12]:
# Portfolio TPV DataFrame
tpv_df = dd.read_csv("/content/drive/MyDrive/Colab Datasets/Stone Data Challenge 2022/raw/portfolio_tpv.csv")

In [13]:
print(type(clientes_df))
print(type(comunicados_df))
print(type(geral_df))
print(type(tpv_df))

<class 'dask.dataframe.core.DataFrame'>
<class 'dask.dataframe.core.DataFrame'>
<class 'dask.dataframe.core.DataFrame'>
<class 'dask.dataframe.core.DataFrame'>


In [16]:
clientes_df.compute().info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 14405 entries, 0 to 14404
Data columns (total 6 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   nr_documento  14405 non-null  object
 1   tipo_empresa  14405 non-null  object
 2   cidade        14405 non-null  object
 3   estado        14405 non-null  object
 4   subsegmento   14405 non-null  object
 5   segmento      14405 non-null  object
dtypes: object(6)
memory usage: 675.4+ KB


In [23]:
comunicados_df.compute().info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 403740 entries, 0 to 403739
Data columns (total 6 columns):
 #   Column            Non-Null Count   Dtype 
---  ------            --------------   ----- 
 0   contrato_id       403740 non-null  object
 1   dt_ref_portfolio  403740 non-null  object
 2   data_acao         403740 non-null  object
 3   tipo_acao         403740 non-null  object
 4   acao              403740 non-null  object
 5   status            403740 non-null  object
dtypes: object(6)
memory usage: 18.5+ MB


In [18]:
geral_df.compute().info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 8061006 entries, 0 to 260050
Data columns (total 22 columns):
 #   Column                      Dtype  
---  ------                      -----  
 0   contrato_id                 object 
 1   dt_ref_portfolio            object 
 2   safra                       object 
 3   nr_documento                object 
 4   status_contrato             object 
 5   dt_contrato                 object 
 6   dt_desembolso               object 
 7   dt_vencimento               object 
 8   dt_wo                       object 
 9   prazo                       float64
 10  vlr_desembolsado            float64
 11  vlr_tarifa                  float64
 12  juros_mes                   float64
 13  juros_diario                float64
 14  perc_retencao               float64
 15  vlr_pgto_realizado          float64
 16  vlr_pgto_esperado           float64
 17  vlr_saldo_devedor           float64
 18  vlr_saldo_devedor_esperado  float64
 19  dsp                   

In [19]:
tpv_df.compute().info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 4408597 entries, 0 to 1101395
Data columns (total 4 columns):
 #   Column          Dtype  
---  ------          -----  
 0   nr_documento    object 
 1   dt_transacao    int64  
 2   qtd_transacoes  int64  
 3   vlr_tpv         float64
dtypes: float64(1), int64(2), object(1)
memory usage: 168.2+ MB


In [30]:
comunicados_df = comunicados_df.astype({'dt_ref_portfolio': 'datetime64', 'data_acao': 'datetime64'})

In [31]:
geral_df = geral_df.astype({
    'dt_ref_portfolio': 'datetime64',
    'safra': 'datetime64',
    'dt_contrato': 'datetime64',
    'dt_desembolso': 'datetime64',
    'dt_vencimento': 'datetime64',
    'dt_wo': 'datetime64',
    'prazo': 'int'
})

In [32]:
tpv_df = tpv_df.astype({'dt_transacao': 'object'})

In [36]:
print(f'DataFrame Portfolio Clientes tem {clientes_df.compute().shape[0]} linhas.')
print(f'DataFrame Portfolio Comunicados tem {comunicados_df.compute().shape[0]} linhas.')
print(f'DataFrame Portfolio Geral tem {geral_df.compute().shape[0]} linhas.')
print(f'DataFrame Portfolio TPV tem {tpv_df.compute().shape[0]} linhas.')

DataFrame Portfolio Clientes tem 14405 linhas.
DataFrame Portfolio Comunicados tem 403740 linhas.
DataFrame Portfolio Geral tem 8061006 linhas.
DataFrame Portfolio TPV tem 4408597 linhas.


In [37]:
clientes_df.head()

,nr_documento,tipo_empresa,cidade,estado,subsegmento,segmento
0,094aaed740bbd7fcddb6c678c18d883a,PJ,Caucaia,CE,Alimentação Rápida,Alimentação
1,eaade7915611fcdcbba747e3a83365c6,PF,Cidade Ocidental,GO,Lojas Diversas,Varejo
2,1f21958fd296f1b04a1f0c2658913856,MEI,Manaus,AM,Alimentação Rápida,Alimentação
3,fcdfe15db4520db1a1fe8b6a72878655,MEI,São Bernardo do Campo,SP,Alimentação Rápida,Alimentação
4,ba57e8837452b5e0114d7e273de039b4,PJ,Foz do Iguaçu,PR,Lojas Diversas,Supermercado/Farmácia


In [38]:
comunicados_df.head()

,contrato_id,dt_ref_portfolio,data_acao,tipo_acao,acao,status
0,0d823ddff1839eb938dccc68586ae365,2021-01-16,2021-01-18,HSM,campanhaobservacao,LIDO
1,c2f8a21671dbaeaa4157e03682c639e1,2021-05-22,2021-05-24,HSM,campanhaobservacao,ENTREGUE
2,e19a222c233d46ff4984489c13aa1b98,2020-09-15,2020-09-15,HSM,campanhaobservacao,NAO ENTREGUE
3,c24fc7630fe5b89928ba9478c2cdd9b2,2021-07-20,2021-07-20,HSM,campanhaobservacao,NAO ENTREGUE
4,0abe9c2f21e941f2bd3b086277e33ebc,2020-12-26,2020-12-28,HSM,campanhaobservacao,ENTREGUE


In [39]:
geral_df.head()

,contrato_id,dt_ref_portfolio,safra,nr_documento,status_contrato,dt_contrato,dt_desembolso,dt_vencimento,dt_wo,prazo,...,juros_mes,juros_diario,perc_retencao,vlr_pgto_realizado,vlr_pgto_esperado,vlr_saldo_devedor,vlr_saldo_devedor_esperado,dsp,dspp,flag_transacao
0,356c02706c8e74b15004bb5964ade6bb,2022-03-01,2020-06-01,b0871d4e0d72afd1d44381e5b2453cb2,Settled,2020-06-24,2020-06-29,2020-12-24,2021-12-24,10,...,0.10788,0.00345,0.24000,0.0,163.00,0.55,0.00,0,0,0
1,1eb0e13f83f336146227ccc9efb08c47,2022-03-01,2020-07-01,938513461b4a4fcd2bf99ffd8f12b9be,Settled,2020-07-26,2020-07-28,2021-10-26,2022-10-26,25,...,0.04788,0.00157,0.08400,0.0,46.87,0.00,0.00,0,0,0
2,8b03675f597974ca96f06648e96af6d4,2022-03-01,2020-08-01,4303bd71ee3fe80aba4efc8dbd6c7d70,Settled,2020-08-13,2020-08-17,2022-02-13,2023-02-13,30,...,0.05388,0.00176,0.08076,0.0,45.57,0.00,0.00,0,0,0
3,4207c25cdd13aba5a05ae348e889d798,2022-03-01,2021-03-01,9c1dc4403f8a4833bd58eea3ee85c917,Active,2021-03-01,2021-03-03,2022-06-01,2023-06-01,25,...,0.04788,0.00157,0.19200,0.0,43.06,9469.47,461.96,20,82,0
4,7dfdca96b6ff0dd844e5e130aee88b30,2022-03-01,2020-11-01,3b6951df3075979c99a4dfd9f93efd38,Active,2020-10-31,2020-11-04,2022-01-31,2023-01-31,25,...,0.04788,0.00157,1.20000,0.0,200.09,57783.55,0.00,134,167,0


In [40]:
tpv_df.head()

,nr_documento,dt_transacao,qtd_transacoes,vlr_tpv
0,7f0464355620bc7c137b3546eb90867a,20210505,4,116.5
1,7f0464355620bc7c137b3546eb90867a,20210506,20,537.0
2,7f0464355620bc7c137b3546eb90867a,20210507,17,440.0
3,7f0464355620bc7c137b3546eb90867a,20210508,29,1328.0
4,7f0464355620bc7c137b3546eb90867a,20210509,35,969.5


#2. Limpeza dos dados

In [41]:
clientes_df.isna().sum().compute()

nr_documento    0
tipo_empresa    0
cidade          0
estado          0
subsegmento     0
segmento        0
dtype: int64

In [42]:
comunicados_df.isna().sum().compute()

contrato_id         0
dt_ref_portfolio    0
data_acao           0
tipo_acao           0
acao                0
status              0
dtype: int64

In [43]:
geral_df.isna().sum().compute()

contrato_id                   0
dt_ref_portfolio              0
safra                         0
nr_documento                  0
status_contrato               0
dt_contrato                   0
dt_desembolso                 0
dt_vencimento                 0
dt_wo                         0
prazo                         0
vlr_desembolsado              0
vlr_tarifa                    0
juros_mes                     0
juros_diario                  0
perc_retencao                 0
vlr_pgto_realizado            0
vlr_pgto_esperado             0
vlr_saldo_devedor             0
vlr_saldo_devedor_esperado    0
dsp                           0
dspp                          0
flag_transacao                0
dtype: int64

In [44]:
tpv_df.isna().sum().compute()

nr_documento      0
dt_transacao      0
qtd_transacoes    0
vlr_tpv           0
dtype: int64

In [47]:
comunicados_df['status'].value_counts().compute()

NAO ENTREGUE    191207
ENTREGUE        137951
LIDO             71380
RESPONDIDO        3202
Name: status, dtype: int64

In [48]:
comunicados_df['tipo_acao'].value_counts().compute()

EMAIL    201870
HSM      201870
Name: tipo_acao, dtype: int64

In [49]:
comunicados_df['acao'].value_counts().compute()

campanhaobservacao        213334
campanhaparcelamento       89926
campanhaboletoquitado      68370
campanhaprenegativacao     21190
campanhanegativacao        10920
Name: acao, dtype: int64